In [1]:
// WRAPPER Vector2D → p5.Vector (Opción 2 - Robusta) con soporte para Verlet

function Vector2D(x, y) {
    if (typeof x === 'object' && x instanceof p5.Vector) {
        this._vec = x.copy();
    } else if (typeof x === 'object' && x instanceof Vector2D) {
        this._vec = x._vec.copy();
    } else {
        this._vec = createVector(x || 0, y || 0);
    }
}

function toP5Vector(arg) {
    if (arg instanceof Vector2D) return arg._vec;
    if (arg instanceof p5.Vector) return arg;
    if (arg && typeof arg.x === 'number' && typeof arg.y === 'number') {
        return createVector(arg.x, arg.y);
    }
    return arg;
}

Vector2D.prototype = {
    get x() { return this._vec.x; },
    set x(val) { this._vec.x = val; },
    get y() { return this._vec.y; },
    set y(val) { this._vec.y = val; },
    
    lengthSquared: function() {
        return this._vec.magSq();
    },
    
    length: function() {
        return this._vec.mag();
    },
    
    clone: function() {
        return new Vector2D(this._vec);
    },
    
    add: function(vec) {
        var result = this._vec.copy().add(toP5Vector(vec));
        return new Vector2D(result);
    },
    
    subtract: function(vec) {
        var result = this._vec.copy().sub(toP5Vector(vec));
        return new Vector2D(result);
    },
    
    dotProduct: function(vec) {
        return this._vec.dot(toP5Vector(vec));
    },
    
    negate: function() {
        this._vec.mult(-1);
        return this;
    },
    
    normalize: function() {
        this._vec.normalize();
        return this.length();
    },
    
    incrementBy: function(vec) {
        this._vec.add(toP5Vector(vec));
        return this;
    },
    
    decrementBy: function(vec) {
        this._vec.sub(toP5Vector(vec));
        return this;
    },
    
    scaleBy: function(k) {
        this._vec.mult(k);
        return this;
    },
    
    // Métodos adicionales para Verlet
    scaleAndAdd: function(vec, scalar) {
        // this += vec * scalar (operación eficiente para Verlet)
        var temp = toP5Vector(vec).copy().mult(scalar);
        this._vec.add(temp);
        return this;
    },
    
    toP5Vector: function() {
        return this._vec.copy();
    }
};

Vector2D.distance = function(vec1, vec2) {
    return vec1._vec.dist(toP5Vector(vec2));
};

Vector2D.angleBetween = function(vec1, vec2) {
    return vec1._vec.angleBetween(toP5Vector(vec2));
};

Vector2D.fromP5Vector = function(p5vec) {
    return new Vector2D(p5vec);
};

console.log("Vector2D wrapper con soporte Verlet cargado");

Vector2D wrapper con soporte Verlet cargado


In [2]:
// VARIABLES GLOBALES DEL SISTEMA

window.pelotas = [];
window.numPelotas = 8;
window.gravedad = null;
window.sueloY = null;
window.restitucion = null;
window.sliderRestitucion = null;

// Parámetro de paso de tiempo para Verlet (dt = 1 en unidades de frame)
window.dt = 1;
window.dt2 = 1; // dt^2
window.medioDt2 = 0.5; // 0.5 * dt^2

console.log("Variables globales inicializadas");

Variables globales inicializadas


In [3]:
// CLASE PELOTA CON INTEGRACIÓN VELOCITY VERLET

function Pelota(x, y, radio, col) {
    // Vectores de estado
    this.pos = new Vector2D(x, y);
    this.vel = new Vector2D(random(-3, 3), random(-2, 2));
    this.acc = new Vector2D(0, 0);
    
    // Para Velocity Verlet: almacenar aceleración anterior
    this.accAnterior = new Vector2D(0, 0);
    
    // Propiedades físicas
    this.radio = radio || random(15, 30);
    this.miColor = col || [random(100, 255), random(100, 255), random(100, 255)];
    this.masa = this.radio / 10;
}

Pelota.prototype = {
    // Calcular fuerzas y aceleración (F = ma → a = F/m)
    calcularAceleracion: function() {
        // Reiniciar aceleración
        this.acc = new Vector2D(0, 0);
        
        // Gravedad: F = mg → a = g (independiente de la masa)
        this.acc.incrementBy(window.gravedad);
        
        // Aquí se pueden agregar más fuerzas (arrastre, viento, etc.)
        // var arrastre = this.vel.clone().scaleBy(-0.001 * this.vel.length());
        // this.acc.incrementBy(arrastre);
    },
    
    // VELOCITY VERLET: Paso de integración
    actualizarVerlet: function() {
        // Guardar aceleración actual como "anterior" para el siguiente paso
        this.accAnterior = this.acc.clone();
        
        // PASO 1: r(t+dt) = r(t) + v(t)*dt + 0.5*a(t)*dt^2
        // En forma vectorial: pos += vel*dt + acc*(0.5*dt^2)
        this.pos.scaleAndAdd(this.vel, window.dt);
        this.pos.scaleAndAdd(this.acc, window.medioDt2);
        
        // PASO 2: Recalcular aceleración a(t+dt) con nueva posición
        this.calcularAceleracion();
        
        // PASO 3: v(t+dt) = v(t) + 0.5*(a(t) + a(t+dt))*dt
        // Promedio de aceleraciones: (accAnterior + acc) / 2
        var accPromedio = this.accAnterior.clone().add(this.acc).scaleBy(0.5);
        this.vel.scaleAndAdd(accPromedio, window.dt);
    },
    
    // Método alternativo: Verlet básico (sin velocidad explícita)
    // No usado aquí, pero incluido para referencia
    actualizarVerletBasico: function(posAnterior) {
        // r(t+dt) = 2r(t) - r(t-dt) + a(t)*dt^2
        var temp = this.pos.clone().scaleBy(2).subtract(posAnterior).add(this.acc.clone().scaleBy(window.dt2));
        return temp;
    },
    
    dibujar: function() {
        fill(this.miColor[0], this.miColor[1], this.miColor[2]);
        noStroke();
        ellipse(this.pos.x, this.pos.y, this.radio * 2);
        
        // Vector velocidad (escalado 3x para visibilidad)
        stroke(255, 0, 0, 150);
        strokeWeight(2);
        line(
            this.pos.x, this.pos.y,
            this.pos.x + this.vel.x * 3,
            this.pos.y + this.vel.y * 3
        );
    },
    
    verificarBordes: function(suelo, e) {
        var restituido = false;
        
        // Colisión con suelo
        if (this.pos.y > suelo - this.radio) {
            this.pos.y = suelo - this.radio;
            this.vel.y *= -e;
            // Fricción de Coulomb simplificada
            this.vel.x *= 0.99;
            restituido = true;
        }
        
        // Techo
        if (this.pos.y < this.radio) {
            this.pos.y = this.radio;
            this.vel.y *= -e;
            restituido = true;
        }
        
        // Pared izquierda
        if (this.pos.x < this.radio) {
            this.pos.x = this.radio;
            this.vel.x *= -e;
            restituido = true;
        }
        
        // Pared derecha
        if (this.pos.x > width - this.radio) {
            this.pos.x = width - this.radio;
            this.vel.x *= -e;
            restituido = true;
        }
        
        // Si hubo colisión, recalcular aceleración para Verlet
        if (restituido) {
            this.calcularAceleracion();
        }
    },
    
    // Energía cinética: K = 0.5 * m * |v|^2
    energiaCinetica: function() {
        return 0.5 * this.masa * this.vel.lengthSquared();
    },
    
    // Energía potencial gravitatoria: U = m * g * y
    // (referencia: y = 0, aumenta hacia abajo en canvas)
    energiaPotencial: function() {
        return this.masa * window.gravedad.y * this.pos.y;
    },
    
    // Energía mecánica total
    energiaTotal: function() {
        return this.energiaCinetica() + this.energiaPotencial();
    }
};

console.log("Clase Pelota con Velocity Verlet cargada");

Clase Pelota con Velocity Verlet cargada


In [4]:
// SETUP Y DRAW CON VELOCITY VERLET

function setup() {
    createCanvas(800, 500);
    
    // Configuración física
    window.gravedad = new Vector2D(0, 0.4);
    window.sueloY = 450;
    window.restitucion = 0.8;
    
    // Actualizar parámetros de tiempo para Verlet
    window.dt = 1;
    window.dt2 = window.dt * window.dt;
    window.medioDt2 = 0.5 * window.dt2;
    
    // Slider de control
    window.sliderRestitucion = createSlider(0.0, 1.2, 0.8, 0.05);
    window.sliderRestitucion.position(20, 20);
    window.sliderRestitucion.style('width', '200px');
    
    // Crear pelotas con condiciones iniciales aleatorias
    for (var i = 0; i < window.numPelotas; i++) {
        var x = random(50, width - 50);
        var y = random(50, 200);
        var pelota = new Pelota(x, y);
        
        // Inicializar aceleración para el primer paso de Verlet
        pelota.calcularAceleracion();
        
        window.pelotas.push(pelota);
    }
    
    console.log("Setup completado - " + window.numPelotas + " pelotas con Velocity Verlet");
}

function draw() {
    background(30);
    
    // Verificación de seguridad
    if (!window.pelotas || window.pelotas.length === 0) {
        fill(255);
        textAlign(CENTER, CENTER);
        textSize(20);
        text("Error: Ejecutar celdas anteriores", width/2, height/2);
        return;
    }
    
    // Actualizar parámetros desde UI
    window.restitucion = window.sliderRestitucion.value();
    var e = window.restitucion;
    
    // Dibujar suelo
    stroke(100, 255, 100);
    strokeWeight(3);
    line(0, window.sueloY, width, window.sueloY);
    
    // Variables para energía total del sistema
    var energiaTotalSistema = 0;
    
    // Actualizar y dibujar cada pelota con Velocity Verlet
    for (var i = 0; i < window.pelotas.length; i++) {
        var p = window.pelotas[i];
        
        // PASO VERLET: Integración
        p.actualizarVerlet();
        
        // Colisiones con bordes (con corrección de aceleración)
        p.verificarBordes(window.sueloY, e);
        
        // Dibujar
        p.dibujar();
        
        // Acumular energía para monitoreo
        energiaTotalSistema += p.energiaTotal();
    }
    
    // Información en pantalla
    fill(255);
    noStroke();
    textAlign(LEFT);
    textSize(14);
    text("PELOTAS CON VELOCITY VERLET", 240, 30);
    text("Restitucion (e): " + e.toFixed(2), 240, 50);
    text("Pelotas: " + window.pelotas.length, 240, 70);
    text("Energia total aprox: " + energiaTotalSistema.toFixed(1), 240, 90);
    text("dt: " + window.dt + " | dt²/2: " + window.medioDt2, 240, 110);
    
    // Instrucciones
    textSize(12);
    fill(200);
    text("Ajusta el slider para cambiar restitucion", 20, 470);
    text("Verlet: reversible y conservativo", 20, 485);
    
    // Reinicio automático si todas están en equilibrio
    var todasDetenidas = true;
    for (var i = 0; i < window.pelotas.length; i++) {
        var p = window.pelotas[i];
        if (Math.abs(p.vel.y) > 0.5 || p.pos.y < window.sueloY - p.radio - 5) {
            todasDetenidas = false;
            break;
        }
    }
    
    if (todasDetenidas) {
        for (var i = 0; i < window.pelotas.length; i++) {
            var p = window.pelotas[i];
            p.pos = new Vector2D(random(50, width - 50), random(50, 150));
            p.vel = new Vector2D(random(-5, 5), random(-2, 2));
            p.acc = new Vector2D(0, 0);
            p.accAnterior = new Vector2D(0, 0);
            p.calcularAceleracion();
        }
    }
}

In [5]:
%show

p5.js seems to have been imported multiple times. Please remove the duplicate import
